# Libraries

In [1]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import scipy
import math
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

import nltk
from nltk.corpus import stopwords

import torch
from torchtext import data
from torchtext import datasets

# Recommend with SVD and Emotions

## Emotion Analysis

Εξάγουμε το συναίσθημα από το input του χρήστη, και βρίσκουμε τα ξενοδοχεία που προσφέρουν αυτό το συναίσθημα περισσότερο.

In [ ]:
df_emotions = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/reviews_with_emotions_full.csv')
df_emotions.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [ ]:
username = input ("Username: ")
password = input ("Password: ")

Username: 1021
Password: 2


In [ ]:
users_comments = df_emotions[df_emotions['UserId'] == int(username)]

In [ ]:
user_positive_comments = users_comments[users_comments['Rating'] > 3]

In [ ]:
already_visited = users_comments["Name"].values

In [ ]:
user_positive_comments = user_positive_comments.groupby("Emotion").count()
user_positive_comments = user_positive_comments.sort_values(by=['Emotion'], ascending = True)

In [ ]:
most_felt_emotion = user_positive_comments.index.values[0]

In [ ]:
possible_options = df_emotions[df_emotions['Emotion'] == most_felt_emotion]

In [ ]:
recommends_df = possible_options.groupby("Name")['Emotion'].count().to_frame()
recommends_df = recommends_df.sort_values(by=['Emotion'], ascending = False)

In [ ]:
possible_recommendations = recommends_df.index.values

In [ ]:
recommendations_from_emotion = []
for i in possible_recommendations:
  if i not in already_visited:
    recommendations_from_emotion.append(i)

## SVD Implementation

In [ ]:
# Δεν έχει σημασία πιο dataset θα χρησιμοποιήσουμε.
# Η μόνη προϋπόθεση είναι να υπάρχει το όνομα του ξενοδοχείου, το UserId και το rating για να γίνει το pivot.
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/airlines_for_keywords_recommendation.csv')

In [ ]:
pivot = pd.pivot_table(df, index='UserId', columns=['Name'], values='Rating').fillna(0)
pivot.head(5)

Name,aegean,aer lingus,aeroloft,air astana,air baltic,air canada,air europa,air france,air mauritius,air transat,american airlines,nippon
UserId,,,,,,,,,,,,
1,1.0,1.0,5.0,1.0,5.0,1.0,1.0,5.0,5.0,2.0,5.0,4.0
2,1.0,1.0,4.0,5.0,1.0,1.0,1.0,5.0,5.0,5.0,1.0,1.0
3,1.0,1.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,4.0,4.0,5.0
4,4.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,5.0,4.0,1.0,5.0
5,5.0,1.0,1.0,5.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
pivot_mat = pivot.values
pivot_mat[:5]

array([[1., 1., 5., 1., 5., 1., 1., 5., 5., 2., 5., 4.],
       [1., 1., 4., 5., 1., 1., 1., 5., 5., 5., 1., 1.],
       [1., 1., 4., 1., 1., 1., 2., 1., 1., 4., 4., 5.],
       [4., 1., 1., 4., 1., 1., 1., 1., 5., 4., 1., 5.],
       [5., 1., 1., 5., 1., 2., 1., 1., 1., 1., 1., 1.]])

In [ ]:
user_id = list(pivot.index)
user_id[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
sparse_matrix = csr_matrix(pivot_mat)
sparse_matrix

<1998x12 sparse matrix of type '<class 'numpy.float64'>'
	with 11411 stored elements in Compressed Sparse Row format>

In [ ]:
factor_n = 7
U, sigma, V = svds(sparse_matrix, k = factor_n)

In [ ]:
print(U.shape)
print(V.shape)
sigma = np.diag(sigma)
print(sigma.shape)

(1998, 7)
(7, 12)
(7, 7)


In [ ]:
pred_rating = np.dot(np.dot(U,sigma),V)
pred_rating[:5]

array([[ 1.47735302,  2.75250629,  5.13021849,  3.14295659,  0.48222386,
         1.13615406,  0.03790617,  3.11405075,  3.42480452,  2.60579717,
         5.0664233 ,  3.57500779],
       [ 1.02059153,  1.75006341,  4.93408857,  4.47959456,  0.51891932,
         1.15812831,  0.29065536,  3.00944896,  5.37559047,  5.15173955,
         0.95975092,  0.87374753],
       [ 1.05016217,  1.53946313,  3.42217619,  1.29487118,  0.12940502,
         0.38080595,  2.18904037,  1.20987891,  1.25647829,  3.9165228 ,
         4.00012046,  5.07824337],
       [ 3.86126431, -0.43338641,  2.47858449,  3.92912507,  0.40470436,
         1.17695601,  0.42916935,  0.66305205,  4.40398124,  4.0398438 ,
         1.0589808 ,  4.88341024],
       [ 4.49196849, -0.07880975,  2.25752835,  2.77384434,  0.2643089 ,
         1.02570313,  0.83660118,  0.59569748,  3.17466365,  0.41493836,
         0.96027173,  1.35531497]])

In [ ]:
pred_rating_n = (pred_rating - pred_rating.min()) / (pred_rating.max() - pred_rating.min())

In [ ]:
pred_rating_n[:5]

array([[0.32538217, 0.47423154, 0.75178323, 0.51980902, 0.20922018,
        0.28555383, 0.15735473, 0.51643483, 0.55270929, 0.4571061 ,
        0.74433638, 0.5702426 ],
       [0.27206415, 0.35721581, 0.72888887, 0.67583553, 0.21350367,
        0.28811889, 0.18685829, 0.5042246 , 0.78042564, 0.75429539,
        0.26496219, 0.25492296],
       [0.27551594, 0.33263232, 0.55240248, 0.30408097, 0.16803544,
        0.19738161, 0.40845784, 0.29415977, 0.29959934, 0.61010784,
        0.61986625, 0.74571614],
       [0.60365749, 0.10234048, 0.44225649, 0.61157891, 0.20017129,
        0.29031666, 0.2030271 , 0.23032836, 0.66700914, 0.62450317,
        0.27654535, 0.72297316],
       [0.67727995, 0.14373041, 0.41645248, 0.47672234, 0.18378285,
        0.27266083, 0.25058685, 0.22246602, 0.52351021, 0.20136591,
        0.26502298, 0.3111366 ]])

In [ ]:
pred_df = pd.DataFrame(pred_rating_n, columns = pivot.columns, index=user_id).transpose()
pred_df.head(15)

,1,2,3,4,5,6,7,8,9,10,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
Name,,,,,,,,,,,,,,,,,,,,,
aegean,0.325382,0.272064,0.275516,0.603657,0.677280,0.418660,0.231277,0.256056,0.641086,0.731083,...,0.152907,0.152907,0.152907,0.152907,0.152907,0.152907,0.152907,0.152907,0.152907,0.152884
aer lingus,0.474232,0.357216,0.332632,0.102340,0.143730,0.361897,0.294207,0.410039,0.307325,0.250733,...,0.155406,0.155406,0.155406,0.155406,0.155406,0.155406,0.155406,0.155406,0.155406,0.157882
aeroloft,0.751783,0.728889,0.552402,0.442256,0.416452,0.659759,0.592546,0.644939,0.638515,0.681140,...,0.151761,0.151761,0.151761,0.151761,0.151761,0.151761,0.151761,0.151761,0.151761,0.150591
air astana,0.519809,0.675836,0.304081,0.611579,0.476722,0.503795,0.617811,0.431762,0.508518,0.703644,...,0.150700,0.150700,0.150700,0.150700,0.150700,0.150700,0.150700,0.150700,0.150700,0.148470
air baltic,0.209220,0.213504,0.168035,0.200171,0.183783,0.190986,0.197293,0.186788,0.190171,0.207354,...,0.154105,0.154105,0.154105,0.154105,0.154105,0.154105,0.154105,0.154105,0.154105,0.155280
air canada,0.285554,0.288119,0.197382,0.290317,0.272661,0.278750,0.330369,0.271709,0.286777,0.348185,...,0.154231,0.154231,0.154231,0.154231,0.154231,0.154231,0.154231,0.154231,0.154231,0.155531
air europa,0.157355,0.186858,0.408458,0.203027,0.250587,0.287517,0.319378,0.322528,0.299863,0.305560,...,0.153295,0.153295,0.153295,0.153295,0.153295,0.153295,0.153295,0.153295,0.153295,0.153660
air france,0.516435,0.504225,0.294160,0.230328,0.222466,0.397656,0.292268,0.368218,0.357445,0.360693,...,0.151678,0.151678,0.151678,0.151678,0.151678,0.151678,0.151678,0.151678,0.151678,0.150427
air mauritius,0.552709,0.780426,0.299599,0.667009,0.523510,0.517737,0.555252,0.393007,0.530252,0.762185,...,0.155401,0.155401,0.155401,0.155401,0.155401,0.155401,0.155401,0.155401,0.155401,0.157872


In [ ]:
def recommend_airlines(user_id ,pred_df, items_df, airlines_to_ignore=[], top_list=20, verbose=False):

        sorted_user_predictions = pred_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        recommendations_df = sorted_user_predictions[~sorted_user_predictions['Name'].isin(airlines_to_ignore)].sort_values(by='recStrength', ascending = False).head(top_list)
        print(recommendations_df)
        return recommendations_df


In [ ]:
def recommender(user_id,pred_df,real_df,df_meta):

    airlines_to_ignore = df[df['UserId'] == str(user_id)]['Name'].values
    recommend = recommend_airlines(user_id, pred_df,real_df, airlines_to_ignore)
    df_user= real_df.loc[real_df['UserId'] == user_id]
    new_df = df_user.merge(recommend, how = 'outer', left_on = 'Name', right_on = 'Name')
    rec_df = new_df.loc[new_df['Rating'].isnull()==True]
    df_rec_t = rec_df.loc[:, ['Name', 'recStrength']]
    df_last = pd.merge(df_rec_t,df_meta[['AirlineId','Name']],on=['Name'], how='left')
    df_last = df_last.drop_duplicates(subset=["Name", "AirlineId"], keep='first')
    df_last = df_last.reset_index(drop=True) 
    return df_last

In [ ]:
airlines = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/airlines.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/ratings.csv')
df_meta = airlines.merge(ratings, how="left", on="AirlineId")
df_meta = df_meta.astype({"UserId": str, "AirlineId": str})
df_meta.head()

,AirlineId,Name,UserId,Review,Rating
0,1,aegean,1,Forgot my baggage,1
1,1,aegean,2,Late or cancelled,1
2,1,aegean,3,Worst experience ever! AVOID AEGEAN!,1
3,1,aegean,4,A great airline,4
4,1,aegean,5,"Accommodating, Lovely Airline",5


In [ ]:
df = df.astype({"UserId": str})
df

,UserId,Name,Review,Rating,Emotion,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count,word_without_stop,positive
0,1,aegean,forgot my baggage,1,neutral,0,forgot my baggage,12,1200,forgot baggage,False
1,105,nippon,like going back in time. seats.,5,neutral,2,like going back in time seats,12,1300,like going back time seats,True
2,112,nippon,okay,3,approval,1,okay,12,1300,okay,True
3,111,nippon,nice airline but seats are not so comfortable,3,disapproval,1,nice airline but seats are not so comfortable,12,1300,nice airline seats comfortable,True
4,110,nippon,i am in love with this airline,5,love,2,i am in love with this airline,12,1300,love airline,True
...,...,...,...,...,...,...,...,...,...,...,...
11406,1422,american airlines,worst airline ever,1,anger,0,worst airline ever,1,1998,worst airline ever,False
11407,1421,american airlines,disappointed!,1,disappointment,0,disappointed,1,1998,disappointed,False
11408,1420,american airlines,worst staff !!!,1,anger,0,worst staff,1,1998,worst staff,False
11409,1419,american airlines,worst travel experience ever,1,fear,0,worst travel experience ever,1,1998,worst travel experience ever,False


In [ ]:
recommendations_from_SVD = recommender(int(username),pred_df,df,df_meta)

             Name  recStrength
5        aeroloft     0.266842
6   air mauritius     0.163462
7      air baltic     0.135673
8      air canada     0.120794
9      air astana     0.111729
10     air france     0.086332
11     aer lingus     0.080526


## Combinign SVD and Emotion Analysis Results

Τώρα, συνδυάζουμε τα recommendations από τις 2 μεθόδους.
Αυτό γίνεται αυξάνοντας το strength που προέρχεται από το SVD, κατά ένα ποσοστό αναλόγως το συναίσθημα που προσφέρει το κάθε εστιατόριο.
Για παράδειγμα, για το πρώτο εστιατόριο που προτείνεται με βάση το συναίσθημα, αυξάνουμε το strength κατά 20%. Για το δεύτερο, το αυξάνουμε κατά 10% ενώ το τρίτο κατά 5%.

In [ ]:
recommendations_from_emotion

['aeroloft',
 'air astana',
 'air mauritius',
 'air france',
 'aer lingus',
 'air canada',
 'air baltic']

In [ ]:
recommendations_from_SVD

,Name,recStrength,AirlineId
0,aeroloft,0.266842,2
1,air mauritius,0.163462,9
2,air baltic,0.135673,5
3,air canada,0.120794,6
4,air astana,0.111729,4
5,air france,0.086332,8
6,aer lingus,0.080526,3


In [ ]:
score_dict = {}

for index, row in recommendations_from_SVD.iterrows():
  name = row['Name']
  print("Name is ", name)
  recStrength = row['recStrength']
  print("recStrength is ", recStrength)
  if(len(recommendations_from_emotion) >= 3):
    if(name == recommendations_from_emotion[0]):
        print('1')
        recStrength = recStrength + 0.2*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
        recStrength = recStrength + 0.1*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[2]):
        recStrength = recStrength + 0.05*recStrength
        print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 2):
    if(name == recommendations_from_emotion[0]):  
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
      recStrength = recStrength + 0.1*recStrength
      print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 1):
    if(name == recommendations_from_emotion[0]):
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
  print('\n')
  score_dict[index] = recStrength    


Name is  aeroloft
recStrength is  0.2668423332953761
1
new recStrength is  0.32021079995445134


Name is  air mauritius
recStrength is  0.16346202598425214
new recStrength is  0.17163512728346475


Name is  air baltic
recStrength is  0.13567338667423973


Name is  air canada
recStrength is  0.12079446220124594


Name is  air astana
recStrength is  0.1117294549918014
new recStrength is  0.12290240049098154


Name is  air france
recStrength is  0.08633150528793639


Name is  aer lingus
recStrength is  0.08052629258996646




In [ ]:
score_dict

{0: 0.32021079995445134,
 1: 0.17163512728346475,
 2: 0.13567338667423973,
 3: 0.12079446220124594,
 4: 0.12290240049098154,
 5: 0.08633150528793639,
 6: 0.08052629258996646}

In [ ]:
import operator
sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_scores

[(0, 0.32021079995445134),
 (1, 0.17163512728346475),
 (2, 0.13567338667423973),
 (4, 0.12290240049098154),
 (3, 0.12079446220124594),
 (5, 0.08633150528793639),
 (6, 0.08052629258996646)]

In [ ]:
counter = 0
#δημιουργία νέου dataframe για την αποθήκευση των recommendations
recommendations = pd.DataFrame(columns=('Name', 'score'))
# τα reviews με τα υψηλότερα cosign similarity
for i in sorted_scores:
  recommendations = recommendations.append({'Name': recommendations_from_SVD.iloc[i[0]]['Name'],
                          'score': i[1]}, ignore_index=True)
            
  counter += 1
  if counter>10:
    break

In [ ]:
recommendations

,Name,score
0,aeroloft,0.320211
1,air mauritius,0.171635
2,air baltic,0.135673
3,air astana,0.122902
4,air canada,0.120794
5,air france,0.086332
6,aer lingus,0.080526
